In [5]:
## imports
import pandas as pd
import numpy as np
import re

In [ ]:
### load raw data from data folder

housing_listings_na = pd.read_csv('../data/listings_NA.csv')
norways_listings = pd.read_csv('../data/realestate.csv')



### clean zillow listings
### remove any columns not related to address, price or area in na listings
na_cols = ['price', 'address', 'area', 'latLong.longitude', 'latLong.latitude']
housing_listings_na = housing_listings_na[na_cols]
housing_listings_na['price'] = housing_listings_na['price'].str.replace('$', '').str.replace(',', '').str.replace('From', '').str.replace('K', '000')
norways_listings['price'] = norways_listings['price'].str.replace('Totalpris:', '').str.replace(' ', '').str.replace(',', '').str.replace('\xa0','')
norways_listings['area'] = norways_listings['area'].str.replace('\xa0','').str.replace('kr', '')



### assign country to each listing in na listings

countries = []
for address in housing_listings_na['address']:
    try:
        if 'Vancouver' in address:
            countries.append('Canada')
        else :
            countries.append('USA')
    except:
        countries.append('USA')
        continue
housing_listings_na['country'] = countries
norways_listings['country'] = 'Norway'

for listing in norways_listings['price']:
    if '-' in listing:
        norways_listings['price'] = norways_listings['price'].str.split('-').str[0]

### remove any rows with non-numeric values in price column
norways_listings = norways_listings[norways_listings['price'].str.isnumeric()]



### convert listings to euros

def price_converter(row):
    if row['country'] == 'USA':
        return float(row['price']) * 0.91
    elif row['country'] == 'Canada':
        return float(row['price']) * 0.67
    else:
        return float(row['price']) * 0.086

housing_listings_na['price'] = housing_listings_na.apply(price_converter, axis=1)
norways_listings['price'] = norways_listings.apply(price_converter, axis=1)

### convert area to square meters in na listings
def area_converter(row):
    return float(row['area']) * 0.092903

housing_listings_na['area'] = housing_listings_na.apply(area_converter, axis=1)

norways_listings['area'] = norways_listings['area'].apply(lambda x: x.split('-')[0] if '-' in x else x).astype(float)


### create price per square meter column
housing_listings_na['price_per_sqm'] = housing_listings_na['price'] / housing_listings_na['area']
norways_listings['price_per_sqm'] = norways_listings['price'] / norways_listings['area']

print(housing_listings_na.head())
print(norways_listings.head())

In [7]:
# save to new cleaned csv files
housing_listings_na.to_csv('../data/clean/listings_NA.csv', index=False)
norways_listings.to_csv('../data/clean/norway_listings.csv', index=False)